#Organizing User Data

###Getting Original User Comments data into a SQL table

In [9]:
userData1488 = pd.read_csv("/Users/alexsutherland/Documents/Programming/Insight/Archive-WeightLoss/6.15.15 - WeightLossRedditBU/Data/Archive/UserComments1488.csv")

In [8]:
userData4000 = pd.read_csv("/Users/alexsutherland/Documents/Programming/Insight/Archive-WeightLoss/6.15.15 - WeightLossRedditBU/Data/Archive/UserComments4157.csv")

Combining these two datasets:

In [10]:
userData = pd.concat([userData1488, userData4000])

In [13]:
userData1488.shape

(398245, 7)

In [14]:
userData4000.shape

(683660, 7)

In [12]:
userData.shape

(1081905, 7)

In [25]:
#userData = userData.drop(['Unnamed: 0'], axis=1)
userData.head()

,score,controversiality,body,subreddit,username,userid
0,1,0,You won't regret it for sure! : P it's uk size...,loseit,sweetpeach206,1
1,1,0,I always tell my self it's better to point eve...,PointsPlus,sweetpeach206,1
2,1,0,Thanks a lot and good luck with your journey!,loseit,sweetpeach206,1
3,1,0,"You don't have to lift weights to lose weight,...",progresspics,sweetpeach206,1
4,2,0,Awsome! You have something to be really proud ...,progresspics,sweetpeach206,1


Put this in a SQL table:

Looking for NaNs

In [35]:
userData.ix[userData.isnull().apply(lambda x: any(x), axis=1),:]

,score,controversiality,body,subreddit,username,userid
7724,1,0,The Schrader family: 2,NaN,NaN,NaN
7725,breakingbad,MasterBlitzkrieg,22,NaN,NaN,NaN
15366,2,0,Really easy with a ceramic rod like you get wi...,NaN,NaN,NaN
15367,NaN,NaN,NaN,NaN,NaN,NaN
15368,knives,familyphotoshoot,53,NaN,NaN,NaN
15374,16,0,I hope the info about the Mirror Knight boss s...,NaN,NaN,NaN
15375,NaN,NaN,NaN,NaN,NaN,NaN
15376,darksouls,familyphotoshoot,53,NaN,NaN,NaN
15380,2,0,Agreed. There's nothing fun about farming upgr...,NaN,NaN,NaN
15381,NaN,NaN,NaN,NaN,NaN,NaN


Removing all rows with at least one NaN

In [39]:
userData = userData.dropna(axis=0)

In [40]:
import redditDataIO


redditDataIO.exportDataToSQL(userData, tableName='rawCommentData')

Index([u'score', u'controversiality', u'body', u'subreddit', u'username', u'userid'], dtype='object')
Exported Data to SQL Table (Name: rawCommentData)


In [4]:
import sys
sys.path.append('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit')
import pandas as pd
import os
os.chdir('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit')
import qgrid

In [42]:
userData = redditDataIO.loadData(tableName = 'rawCommentData')

In [44]:
qgrid.show_grid(userData.head(30))

In [45]:
userData.columns

Index([u'index', u'score', u'controversiality', u'body', u'subreddit', u'username', u'userid'], dtype='object')

Trying to get the counts of comments at each subreddit using groupby:

In [46]:
userData.groupby(by=['username','subreddit']).count().head()

index  score  controversiality  body  userid
username       subreddit                                                 
-AroundTheFur- pics             14     14                14    14      14
               progresspics     11     11                11    11      11
               sloths            6      6                 6     6       6
-BeatrixKiddo- progresspics     21     21                21    21      21
-ClarkNova-    195               3      3                 3     3       3

This works but does not create a "semi-dummy" list that is desired. Ideally, the columns would be each subreddit and the value would be how many posts each user (rows) have posted there.

We can try crosstab to see if this has the desired output:

In [47]:
crosstabUserData = pd.crosstab(userData['username'], userData['subreddit'], margins=True)
crosstabUserData.head()

subreddit,100DaysofKeto,100pushups,10kchallenge,1200isplenty,1200isplentyketo,12weeksfromtoday,18_19,1911,195,1950s,...,zenfone2,zenfone2photos,zennebaa,zerocarb,zombies,zooeydeschanel,zoology,zumba,zyzz,All
username,,,,,,,,,,,,,,,,,,,,,
-AroundTheFur-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,31
-BeatrixKiddo-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,21
-ClarkNova-,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,386
-nekkidprogg-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
0verstim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,150


Cross-tabulation worked well. 

Now, we need to get the count of comments at fitness related subreddits.

We start by creating a Series of fitness related subreddits:

In [48]:
fitnessSeries = pd.read_csv('Data/fitnessSubreddits.csv')['Subreddits']
print fitnessSeries

0            100pushups
1            90daysgoal
2              athleanx
3                 b210k
4             bodybeast
5          bodybuilding
6     bodyweightfitness
7          briskwalking
8                  BTFC
9             BulkOrCut
10             crossfit
11                 c25k
12                cutit
13        cuttingweight
14                 eood
...
222         fitnessdeals
223        fitnessvideos
224          fittitbuddy
225            formcheck
226         frugalsports
227    frugalsupplements
228               health
229                 iron
230         liftingmusic
231        LiftingVideos
232     runninglifestyle
233         runningmusic
234        runningroutes
235      supplementdeals
236       ThinYourCloset
Name: Subreddits, Length: 237, dtype: object


Let's try to select a subset of subreddits for all users, and get the sum of the cols:

In [49]:
crosstabUserData[['progresspics','crossfit','loseit']].head()

subreddit,progresspics,crossfit,loseit
username,,,
-AroundTheFur-,11,0,0
-BeatrixKiddo-,21,0,0
-ClarkNova-,4,0,13
-nekkidprogg-,12,0,0
0verstim,0,0,1


This works well. Now, we need to make sure that both the fitnessList and crosstabUserData columns are lowercase:

In [50]:
fitnessSeries_lower = fitnessSeries.map(lambda x: x.lower())
print fitnessSeries_lower.head()

0    100pushups
1    90daysgoal
2      athleanx
3         b210k
4     bodybeast
Name: Subreddits, dtype: object


In [51]:
crosstabUserData.columns = pd.Series(crosstabUserData.columns.values).map(lambda x: x.lower())
print crosstabUserData.head()

                100daysofketo  100pushups  10kchallenge  1200isplenty  \
username                                                                
-AroundTheFur-              0           0             0             0   
-BeatrixKiddo-              0           0             0             0   
-ClarkNova-                 0           0             0             0   
-nekkidprogg-               0           0             0             0   
0verstim                    0           0             0             0   

                1200isplentyketo  12weeksfromtoday  18_19  1911  195  1950s  \
username                                                                      
-AroundTheFur-                 0                 0      0     0    0      0   
-BeatrixKiddo-                 0                 0      0     0    0      0   
-ClarkNova-                    0                 0      0     0    3      0   
-nekkidprogg-                  0                 0      0     0    0      0   
0verstim      

Now, both of the subreddit lists are lowercase, so we can begin comparing them.

First, we need to make a list of fitness subreddits, rather than a Series:

In [52]:
fitnessList = fitnessSeries_lower.tolist()
print fitnessList[0:5]

['100pushups', '90daysgoal', 'athleanx', 'b210k', 'bodybeast']


Now, we can get the fitness related columns from the crosstabUserData:

In [53]:
crosstabUserData[fitnessList[0:5]]

KeyError: "['athleanx'] not in index"

There's an error because the fitnessList has some Subreddits that the crosstabUserData does not have. 

One possiblility is to add columns for all the fitnessList Subreddits.

We start this by merging the fitnessList and crosstabUserData.columns:

In [54]:
fullSubredditList = fitnessList + crosstabUserData.columns.tolist()
fullSubredditSeries = pd.Series(fullSubredditList)

fullSubredditListUnqiues = fullSubredditSeries.unique()
print fullSubredditListUnqiues[0:5]

['100pushups' '90daysgoal' 'athleanx' 'b210k' 'bodybeast']


We now add all of these columns to the crosstabUserData dataframe by reindexing:

In [60]:
crosstabUserDataAllSubreddits = crosstabUserData.reindex(columns=fullSubredditListUnqiues, fill_value=0)
crosstabUserDataAllSubreddits.ix[200:225,['1200isplenty','askreddit']]

,1200isplenty,askreddit
username,,
Boredivist,0,34
BornToBus93,0,62
BossFoundMyAccount,0,7
Boughner,0,13
BreakingBob,0,25
BredToUpset,0,7
Breetannica,0,1
Brewers24,0,0
BrianMatthewAnthony,0,3


Now that we have a unique list of subreddits, we can isolate these columns from crosstabUserData:

In [61]:
crosstabUserDataFitness = crosstabUserDataAllSubreddits[fitnessList]

crosstabUserDataFitness.head(10)

,100pushups,90daysgoal,athleanx,b210k,bodybeast,bodybuilding,bodyweightfitness,briskwalking,btfc,bulkorcut,...,frugalsupplements,health,iron,liftingmusic,liftingvideos,runninglifestyle,runningmusic,runningroutes,supplementdeals,thinyourcloset
username,,,,,,,,,,,,,,,,,,,,,
-AroundTheFur-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-BeatrixKiddo-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-ClarkNova-,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-nekkidprogg-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0verstim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109epz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1122boogiewoogieave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11235813213455away,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


These columns are then summed up row-wise to obtain the total number of comments left on fitness subreddits:

In [62]:
userFitnessCommentSeries = crosstabUserDataFitness.sum(axis=1)
userFitnessCommentSeries.head()

username
-AroundTheFur-    11
-BeatrixKiddo-    21
-ClarkNova-       21
-nekkidprogg-     13
0verstim           2
dtype: int64

We convert the userFitnessCommentSeries to a dataframe:

In [63]:
userFitnessCommentDataFrame = pd.DataFrame(userFitnessCommentSeries)
userFitnessCommentDataFrame.columns = ['fitnessCommentCount']
userFitnessCommentDataFrame.head()

,fitnessCommentCount
username,
-AroundTheFur-,11
-BeatrixKiddo-,21
-ClarkNova-,21
-nekkidprogg-,13
0verstim,2


We now add the total number of comments the user has posted:

In [66]:
userFitnessCommentDataFrame['totalCommentCount'] = crosstabUserData['all']
userFitnessCommentDataFrame.head(50)

,fitnessCommentCount,totalCommentCount
username,,
-AroundTheFur-,11,31
-BeatrixKiddo-,21,21
-ClarkNova-,21,386
-nekkidprogg-,13,13
0verstim,2,150
109epz,15,15
110100,4,842
1122boogiewoogieave,28,92
11235813213455away,49,475


Finally, we can calculate the rate of commenting at fitness related subreddits:

In [67]:
userFitnessCommentDataFrame['fitnessCommentFraction'] = userFitnessCommentDataFrame['fitnessCommentCount'] / userFitnessCommentDataFrame['totalCommentCount']
userFitnessCommentDataFrame.head()

,fitnessCommentCount,totalCommentCount,fitnessCommentFraction
username,,,
-AroundTheFur-,11,31,0.354839
-BeatrixKiddo-,21,21,1.000000
-ClarkNova-,21,386,0.054404
-nekkidprogg-,13,13,1.000000
0verstim,2,150,0.013333


Outputting this to a sql table:

In [69]:
redditDataIO.exportDataToSQL(userFitnessCommentDataFrame.reset_index(), tableName='userFitnessCommentData')

Index([u'username', u'fitnessCommentCount', u'totalCommentCount', u'fitnessCommentFraction'], dtype='object')
Exported Data to SQL Table (Name: userFitnessCommentData)
